In [1]:
import os
import glob as glob
import pandas as pd
import numpy as np
from events_prep import *

In [2]:
data_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/raw/'
out_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep/'

files = np.sort(os.listdir(data_dir))

for sub in files:
    file_path = f'{data_dir}{sub}/{sub}_surveyMedley.csv'
    if not os.path.exists(file_path):
        print(f'Survey data not exist for sub-{sub}')
    else:
        file_name = os.path.basename(file_path)
        df = pd.read_csv(file_path)
        
        start_time = df.query('trial_id == "fmri_trigger_wait"').iloc[-1]['time_elapsed']
        df.time_elapsed -= start_time
        
        # correct start time for problematic scans
        df.time_elapsed -= get_timing_correction(file_name)
        df_clean = create_survey_event(df)
        df_clean = df_clean.dropna(axis=0).reset_index(drop=True)
        df_clean['trial_type'] = df_clean['item_text'].map(get_items_order())
        df_clean['trial_no'] = np.arange(1,41)
        
        #events = pd.DataFrame(df_clean, columns=['onset', 'trial_type'])
        #events['duration'] = 3.982
        
        new_dir = f'{out_dir}sub-{sub}'
        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
            
        #events.to_csv(f'{new_dir}/sub-{sub}_surveyMedley_events.csv', index = False)
        df_clean.to_csv(f'{new_dir}/sub-{sub}_surveyMedley_clean.csv', index = False)

/home/finc/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/finc/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Survey data not exist for sub-s144
Survey data not exist for sub-s533
Survey data not exist for sub-s600
Survey data not exist for sub-s603
Survey data not exist for sub-s620
Survey data not exist for sub-s639
Survey data not exist for sub-s640


In [3]:
 df_clean.tail()

,movement_onset,onset,response_time,duration,rt,subject,possible_responses,survey,item_responses,item_coding,item_text,coded_response,junk,trial_type,trial_no
35,352.813,351.389,-3.785222,8.5,1424,s650,"66,89,71,82,77",upps,"66,89,71,82",forward,"When I get really happy about something, I ten...",3.0,False,Q34,36
36,364.419,360.460,-1.250222,8.5,3959,s650,"66,89,71,82,77",future_time,"66,89,71,82,77",forward,My future seems infinite to me.,3.0,False,Q26,37
37,370.618,370.619,-5.210222,8.5,-1,s650,"66,89,71,82,77",upps,"66,89,71,82",forward,"Sometimes when I feel bad, I can't seem to sto...",0.0,True,Q32,38
38,380.505,380.506,-5.210222,8.5,-1,s650,"66,89,71,82,77",brief,"66,89,71,82,77",reverse,I wish I had more self-discipline.,0.0,True,Q15,39
39,390.187,390.188,-5.210222,8.5,-1,s650,"66,89,71,82,77",impulsive_venture,"66,89",forward,Would you enjoy the sensation of skiing very f...,0.0,True,Q40,40
